<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Quantization_for_Edge_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from transformers import LongformerForSequenceClassification, LongformerTokenizer

class EnergyBasedOOD:
    def __init__(self, model):
        self.model = model

    def compute_energy(self, logits):
        return -torch.logsumexp(logits, dim=1)

    def detect_ood(self, input_ids, attention_mask, threshold):
        # Ensure the model is in evaluation mode
        self.model.eval()

        # Disable gradient computation for inference
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Compute the energy scores
        energy = self.compute_energy(logits)

        # Return OOD detection results
        return energy > threshold

# Initialize tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

# Initialize the model for sequence classification
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2)

# Quantize the model dynamically
quantized_model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

# Initialize the OOD detector with the quantized model
ood_detector = EnergyBasedOOD(quantized_model)

# Dummy input data (Ensure this is a tensor of the correct shape)
text_data = ["This is a sample text for model explanation."]
inputs = tokenizer(text_data, return_tensors="pt", truncation=True, padding=True, max_length=64)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Define a threshold for OOD detection
threshold = 0.5

# Detect OOD
ood_results = ood_detector.detect_ood(input_ids, attention_mask, threshold)
print("OOD Detection Results:", ood_results)